## Import packages

In [1]:
import pyomo
import pyomo.opt
import pyomo.environ as pyo
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
n_time = 24
country = "Greece"; # Possible options: Portugal, Slovenia, Greece, Denmark, this if apply only to Slovenia and Denmark, because both countries have biomass 

In [3]:
def _auxDictionary(a):
    temp_dictionary = {}
    if len(a.shape) == 3:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                for dim2 in np.arange(a.shape[2]):
                    temp_dictionary[(dim0+1, dim1+1, dim2+1)] = a[dim0, dim1, dim2]
    elif len(a.shape) == 2:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                temp_dictionary[(dim0+1, dim1+1)] = a[dim0, dim1]
    else:
        for dim0 in np.arange(a.shape[0]):
            temp_dictionary[(dim0+1)] = a[dim0]
    return temp_dictionary
#temp_dict1 = _auxDictionary(loadLimit)

## Data

In [4]:
data = {}
data['Inputs'] = pd.read_csv('Inputs.csv')
data['EVs_Inputs'] = pd.read_csv('EVs_Inputs.csv')
data['alpha'] = pd.read_csv('alpha.csv')
data['PchmaxEV'] = pd.read_csv('PchmaxEV.csv')
data['S'] = pd.read_csv('S.csv')

In [5]:
n_evs = data['EVs_Inputs']['Esoc'].size

## Start Time

In [6]:
from datetime import datetime

now = datetime.now()

start_time = now.strftime("%H:%M:%S")
print("Start Time =", start_time)

Start Time = 00:29:09
Start Time = 00:29:09


## Sets

In [7]:
model = pyo.ConcreteModel()

model.ev = pyo.Set(initialize = np.arange(1, n_evs + 1))
model.t = pyo.Set(initialize = np.arange(1, n_time + 1))


## Parameters

In [8]:
model.EEVmax = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,2]))
model.EEVmin = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,1]))
model.ESoc = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,0]))
model.dT = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,0]))
model.cDA = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,1]))
model.Pl = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,2]))
model.Php = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,3]))
model.Pb = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,4]))
model.Pimp = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,5]))
model.Ppv = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,6]))
model.Pwind = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,7]))
model.Phid = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,8]))
model.Pbio = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,9]))
model.PchmaxEV = pyo.Param(model.ev, model.t, initialize =_auxDictionary(data['PchmaxEV'].to_numpy()))
model.S = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['S'].to_numpy()))
model.alpha = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['alpha'].to_numpy()))
model.RealHour = pyo.Param(model.t, initialize=_auxDictionary(data['Inputs'].to_numpy()[:,9]))
model.penalty1 = 1000
model.penalty2 = 1000
model.penalty3 = 0.6
model.Etrip = pyo.Param(model.ev, initialize=_auxDictionary(data['EVs_Inputs'].to_numpy()[:,3]))
model.n = 0.95
model.m = 0.6
model.factor = 1

## Variables

In [9]:
model.PEV = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.EEV = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)
model.Etriprelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Eminsocrelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Etripn = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)
model.Pgen = pyo.Var(model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Pres = pyo.Var(model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.PminAux = pyo.Var(domain = pyo.NonNegativeReals, initialize = 0)
model.Pex = pyo.Var(model.t, domain = pyo.NonNegativeReals, initialize = 0)

## Constraints

In [10]:
def _balance_etripn(m,ev,t): 
    return m.Etripn[ev,t] == m.Etrip[ev]*m.S[ev,t]/(sum([m.S[ev,k] for k in np.arange(1, n_time + 1)]))
model.balance_etripn = pyo.Constraint(model.ev, model.t, rule = _balance_etripn)

def _balance_energy_EVS3(m,ev,t): 
    if t == 24:
        return m.EEV[ev,t] + m.Etriprelax[ev,t] >= m.EEVmax[ev]*m.m
    return pyo.Constraint.Skip
model.balance_energy_EVS3 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS3)

def _balance_energy_EVS(m,ev,t): 
    if t == 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.ESoc[ev] + m.PEV[ev,t]*m.dT[t]*m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS)

def _balance_energy_EVS2(m,ev,t): 
    if t > 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.EEV[ev,t-1] + m.PEV[ev,t]*m.dT[t]*m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS2 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS2)

def _RES_balance(m,t):
    return m.Pres[t] == m.Pwind[t] + m.Ppv[t] + m.Phid[t] + m.Pbio[t]
model.RES_balance = pyo.Constraint(model.t, rule = _RES_balance)

def _total_balance(m,t):
    #if (country == "Slovenia"):
        #return m.Pgen[t] + m.Pres[t] + m.Pimp[t] + m.Pb[t] - m.Pex[t]== m.Pl[t] + sum(m.PEV[ev, t] for ev in np.arange(1, n_evs + 1))
    #else:
        return m.Pgen[t] + m.Pres[t] + m.Pimp[t] - m.Pex[t] == m.Pl[t] + m.Pb[t] + sum(m.PEV[ev, t] for ev in np.arange(1, n_evs + 1))
model.total_balance = pyo.Constraint(model.t, rule = _total_balance)

def _power_charging_limit1(m,ev,t): 
    return m.PEV[ev,t] >= 0
model.power_charging_limit1 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit1)

def _power_charging_limit2(m,ev,t): 
    return m.PEV[ev,t] <= m.alpha[ev,t]*m.PchmaxEV[ev,t]*m.factor
model.power_charging_limit2 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit2)

def _energy_limits_EVS_1(m,ev,t): 
    return m.EEVmin[ev] <= m.EEV[ev,t] + m.Eminsocrelax[ev,t]
    #return m.EEVmin[ev] <= m.EEV[ev,t]
model.energy_limits_EVS_1 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_1)

def _energy_limits_EVS_2(m,ev,t): 
    return m.EEV[ev,t] <= m.EEVmax[ev]
model.energy_limits_EVS_2 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_2)

def _pmin_aux(m, t):
    return m.PminAux <= m.Pl[t] + sum(m.PEV[ev, t] for ev in np.arange(1, n_evs + 1))
model.pmin_aux = pyo.Constraint(model.t, rule = _pmin_aux)


## Objective function

In [11]:
def _FOag(m):
    return sum(m.Pgen[t] for t in np.arange(1, n_time + 1)) - m.PminAux*0.01 + sum([m.Etriprelax[ev,t]*m.penalty1 + m.Eminsocrelax[ev,t]*m.penalty2 for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

model.FOag = pyo.Objective(rule = _FOag, sense = pyo.minimize)

## Solve the model

In [12]:
from pyomo.opt import SolverFactory

model.write('res_V4_EC.lp',  io_options={'symbolic_solver_labels': True})

# Create a solver
#opt = pyo.SolverFactory('cbc', executable='C:/Program Files/Cbc-releases.2.10.8-x86_64-w64-mingw64/bin/cbc.exe')

opt = pyo.SolverFactory('cplex', executable='C:/Program Files/IBM/ILOG/CPLEX_Studio129/cplex/bin/x64_win64/cplex.exe')
opt.options['LogFile'] = 'res_V4_EC.log'

#opt = pyo.SolverFactory('ipopt', executable='C:/Program Files/Ipopt-3.11.1-win64-intel13.1/bin/ipopt.exe')
#opt.options['print_level'] = 12
#opt.options['output_file'] = "res_V5_EC.log"

results = opt.solve(model)#, tee=True)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: tmp8j_b7xgf
  Lower bound: 622777211.4368602
  Upper bound: 622777211.4368602
  Number of objectives: 1
  Number of constraints: 435073
  Number of variables: 360074
  Number of nonzeros: 939121
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: 4.72
  Termination condition: optimal
  Termination message: Dual simplex - Optimal\x3a Objective = 6.2277721144e+08
  Error rc: 0
  Time: 7.161240577697754
# ----------------------------------------------------------
#   Solution Information
# ---------------

## Objective Function Value

In [13]:
pyo.value(model.FOag)

622777211.4368604

622777211.4368604

## End Time

In [14]:
now = datetime.now()

end_time = now.strftime("%H:%M:%S")
print("End Time =", end_time)
print("Dif: {}".format(datetime.strptime(end_time, "%H:%M:%S") - datetime.strptime(start_time, "%H:%M:%S")))

End Time = 00:31:03
Dif: 0:01:54
End Time = 00:31:03
Dif: 0:01:54


## Cost Value

In [15]:
def ext_pyomo_vals(vals):
    # make a pd.Series from each
    s = pd.Series(vals.extract_values(),
                  index=vals.extract_values().keys())
    # if the series is multi-indexed we need to unstack it...
    if type(s.index[0]) == tuple:    # it is multi-indexed
        s = s.unstack(level=1)
    else:
        # force transition from Series -> df
        s = pd.DataFrame(s)
    return s

In [16]:
PEV_df = ext_pyomo_vals(model.PEV)
dT_df = ext_pyomo_vals(model.dT)
cDA_df = ext_pyomo_vals(model.cDA)
EEV_df = ext_pyomo_vals(model.EEV)

charge_cost = sum([PEV_df[t][ev]*dT_df[0][t]*cDA_df[0][t]
                   for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

print('Charge cost: {}'.format(charge_cost))

Charge cost: 3574733.024043868
Charge cost: 3574733.024043868


## Results

In [17]:
print("Total Charge: {}".format(np.sum(PEV_df.to_numpy())))

Total Charge: 21893970.90015906
Total Charge: 21893970.90015906


In [18]:
EEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,42386.148255,42386.148255,42581.555235,42581.555235,46001.555235,46001.555235,46001.555235,49421.555235,49421.555235,50227.703490,...,47613.851745,47613.851745,47613.851745,47613.851745,47613.851745,47613.851745,47613.851745,47613.851745,47613.851745,45000.0
2,9960.000000,9960.000000,9960.000000,9960.000000,9960.000000,9960.000000,9960.000000,9960.000000,9960.000000,9960.000000,...,10611.876240,10611.876240,10611.876240,10611.876240,10611.876240,10611.876240,10611.876240,10611.876240,10611.876240,9960.0
3,40200.000000,40200.000000,40200.000000,40200.000000,40200.000000,47040.000000,50835.300800,50835.300800,50835.300800,47290.200533,...,43745.100267,43745.100267,43745.100267,43745.100267,43745.100267,43745.100267,43745.100267,43745.100267,43745.100267,40200.0
4,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,...,42852.739320,42852.739320,42852.739320,42852.739320,47147.260680,47147.260680,47147.260680,47147.260680,47147.260680,45000.0
5,46200.000000,46200.000000,46200.000000,46200.000000,46200.000000,46200.000000,46200.000000,46720.845840,46720.845840,46720.845840,...,46460.422920,46460.422920,46460.422920,46460.422920,46460.422920,46460.422920,46460.422920,46460.422920,46460.422920,46200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,...,42223.793400,42223.793400,42223.793400,42223.793400,42223.793400,42223.793400,42223.793400,42223.793400,47776.206600,45000.0
2997,43631.005300,43631.005300,43631.005300,43631.005300,43631.005300,43631.005300,43631.005300,43631.005300,43631.005300,43631.005300,...,42262.010600,42262.010600,42262.010600,42262.010600,42262.010600,42262.010600,42262.010600,42262.010600,45000.000000,45000.0
2998,42000.000000,42000.000000,42000.000000,42000.000000,48840.000000,48840.000000,54034.929120,54034.929120,60874.929120,60874.929120,...,55482.092229,52785.673783,52785.673783,50089.255337,50089.255337,50089.255337,47392.836891,44696.418446,42000.000000,42000.0
2999,43867.405980,43867.405980,43867.405980,43867.405980,43867.405980,43867.405980,43867.405980,43867.405980,43867.405980,43867.405980,...,42734.811960,42734.811960,42734.811960,42734.811960,42734.811960,42734.811960,42734.811960,42734.811960,42734.811960,45000.0


,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,42386.148255,42386.148255,42581.555235,42581.555235,46001.555235,46001.555235,46001.555235,49421.555235,49421.555235,50227.703490,...,47613.851745,47613.851745,47613.851745,47613.851745,47613.851745,47613.851745,47613.851745,47613.851745,47613.851745,45000.0
2,9960.000000,9960.000000,9960.000000,9960.000000,9960.000000,9960.000000,9960.000000,9960.000000,9960.000000,9960.000000,...,10611.876240,10611.876240,10611.876240,10611.876240,10611.876240,10611.876240,10611.876240,10611.876240,10611.876240,9960.0
3,40200.000000,40200.000000,40200.000000,40200.000000,40200.000000,47040.000000,50835.300800,50835.300800,50835.300800,47290.200533,...,43745.100267,43745.100267,43745.100267,43745.100267,43745.100267,43745.100267,43745.100267,43745.100267,43745.100267,40200.0
4,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,...,42852.739320,42852.739320,42852.739320,42852.739320,47147.260680,47147.260680,47147.260680,47147.260680,47147.260680,45000.0
5,46200.000000,46200.000000,46200.000000,46200.000000,46200.000000,46200.000000,46200.000000,46720.845840,46720.845840,46720.845840,...,46460.422920,46460.422920,46460.422920,46460.422920,46460.422920,46460.422920,46460.422920,46460.422920,46460.422920,46200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,...,42223.793400,42223.793400,42223.793400,42223.793400,42223.793400,42223.793400,42223.793400,42223.793400,47776.206600,45000.0
2997,43631.005300,43631.005300,43631.005300,43631.005300,43631.005300,43631.005300,43631.005300,43631.005300,43631.005300,43631.005300,...,42262.010600,42262.010600,42262.010600,42262.010600,42262.010600,42262.010600,42262.010600,42262.010600,45000.000000,45000.0
2998,42000.000000,42000.000000,42000.000000,42000.000000,48840.000000,48840.000000,54034.929120,54034.929120,60874.929120,60874.929120,...,55482.092229,52785.673783,52785.673783,50089.255337,50089.255337,50089.255337,47392.836891,44696.418446,42000.000000,42000.0
2999,43867.405980,43867.405980,43867.405980,43867.405980,43867.405980,43867.405980,43867.405980,43867.405980,43867.405980,43867.405980,...,42734.811960,42734.811960,42734.811960,42734.811960,42734.811960,42734.811960,42734.811960,42734.811960,42734.811960,45000.0


In [19]:
PEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,0.0,0.0,205.691558,0.0,3600.0,0.0,0.000000,3600.000000,0.0,3600.000000,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.000000
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.000000
3,0.0,0.0,0.000000,0.0,0.0,7200.0,3995.053474,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.000000
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,4520.5488,0.0,0.0,0.0,0.000000,0.000000
5,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,548.258779,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,5844.645474,0.000000
2997,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,2882.094105,0.000000
2998,0.0,0.0,0.000000,0.0,7200.0,0.0,5468.346442,0.000000,7200.0,0.000000,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.000000
2999,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,2384.408463


,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,0.0,0.0,205.691558,0.0,3600.0,0.0,0.000000,3600.000000,0.0,3600.000000,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.000000
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.000000
3,0.0,0.0,0.000000,0.0,0.0,7200.0,3995.053474,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.000000
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,4520.5488,0.0,0.0,0.0,0.000000,0.000000
5,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,548.258779,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,5844.645474,0.000000
2997,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,2882.094105,0.000000
2998,0.0,0.0,0.000000,0.0,7200.0,0.0,5468.346442,0.000000,7200.0,0.000000,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.000000
2999,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,2384.408463


In [20]:
PEV_df.sum()

1     3.170878e+03
2     2.854353e+04
3     3.008623e+05
4     6.341996e+05
5     1.382990e+06
6     1.301194e+06
7     1.378186e+06
8     1.445632e+06
9     1.477909e+06
10    1.460056e+06
11    1.249916e+06
12    7.743572e+05
13    1.615230e+05
14    1.030196e+06
15    2.073637e+05
16    3.647842e+04
17    4.668386e+05
18    3.233879e+05
19    3.975597e+05
20    1.465662e+06
21    1.443157e+06
22    1.398301e+06
23    2.065057e+06
24    1.461432e+06
dtype: float64

1     3.170878e+03
2     2.854353e+04
3     3.008623e+05
4     6.341996e+05
5     1.382990e+06
6     1.301194e+06
7     1.378186e+06
8     1.445632e+06
9     1.477909e+06
10    1.460056e+06
11    1.249916e+06
12    7.743572e+05
13    1.615230e+05
14    1.030196e+06
15    2.073637e+05
16    3.647842e+04
17    4.668386e+05
18    3.233879e+05
19    3.975597e+05
20    1.465662e+06
21    1.443157e+06
22    1.398301e+06
23    2.065057e+06
24    1.461432e+06
dtype: float64

In [21]:
Pgen_df = ext_pyomo_vals(model.Pgen)

Pgen_df

,0
1,4.298768e+05
2,2.092353e+05
3,6.073333e+03
4,2.504318e+05
5,0.000000e+00
6,0.000000e+00
7,2.940650e+05
8,3.349024e+05
9,3.463874e+05
10,3.193668e+05


,0
1,4.298768e+05
2,2.092353e+05
3,6.073333e+03
4,2.504318e+05
5,0.000000e+00
6,0.000000e+00
7,2.940650e+05
8,3.349024e+05
9,3.463874e+05
10,3.193668e+05


In [22]:
Pres_df = ext_pyomo_vals(model.Pres)

Pres_df

,0
1,6.085227e+06
2,5.948526e+06
3,6.151688e+06
4,5.907329e+06
5,6.545075e+06
6,6.252979e+06
7,5.863696e+06
8,5.822859e+06
9,5.811374e+06
10,5.838394e+06


,0
1,6.085227e+06
2,5.948526e+06
3,6.151688e+06
4,5.907329e+06
5,6.545075e+06
6,6.252979e+06
7,5.863696e+06
8,5.822859e+06
9,5.811374e+06
10,5.838394e+06


In [23]:
PminAux_df = ext_pyomo_vals(model.PminAux)

PminAux_df

,0
None,6.157761e+06


,0
None,6.157761e+06


In [24]:
Etriprelax_df = ext_pyomo_vals(model.Etriprelax)

In [25]:
Etriprelax_df.sum()

1          0.000000
2          0.000000
3          0.000000
4          0.000000
5          0.000000
6          0.000000
7          0.000000
8          0.000000
9          0.000000
10         0.000000
11         0.000000
12         0.000000
13      2143.550786
14     11719.679421
15     12823.707006
16     65117.304452
17     52771.664291
18    113836.770233
19     42179.246758
20     37857.472378
21     59245.349728
22     10502.340942
23      4926.172590
24    193628.721652
dtype: float64

1          0.000000
2          0.000000
3          0.000000
4          0.000000
5          0.000000
6          0.000000
7          0.000000
8          0.000000
9          0.000000
10         0.000000
11         0.000000
12         0.000000
13      2143.550786
14     11719.679421
15     12823.707006
16     65117.304452
17     52771.664291
18    113836.770233
19     42179.246758
20     37857.472378
21     59245.349728
22     10502.340942
23      4926.172590
24    193628.721652
dtype: float64

In [26]:
Eminsocrelax_df = ext_pyomo_vals(model.Eminsocrelax)

In [27]:
Eminsocrelax_df.sum()

1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
dtype: float64

1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
dtype: float64

In [28]:
Etripn_df = ext_pyomo_vals(model.Etripn)

In [29]:
Etripn_df.sum()

1     1.673788e+06
2     2.896256e+04
3     0.000000e+00
4     0.000000e+00
5     0.000000e+00
6     0.000000e+00
7     0.000000e+00
8     0.000000e+00
9     0.000000e+00
10    5.465601e+05
11    1.315866e+06
12    0.000000e+00
13    8.313551e+05
14    3.272904e+06
15    1.418766e+06
16    1.735838e+06
17    5.656545e+05
18    1.100017e+06
19    3.648812e+05
20    1.147978e+06
21    1.388378e+06
22    1.200184e+06
23    1.404881e+06
24    3.603640e+06
dtype: float64

1     1.673788e+06
2     2.896256e+04
3     0.000000e+00
4     0.000000e+00
5     0.000000e+00
6     0.000000e+00
7     0.000000e+00
8     0.000000e+00
9     0.000000e+00
10    5.465601e+05
11    1.315866e+06
12    0.000000e+00
13    8.313551e+05
14    3.272904e+06
15    1.418766e+06
16    1.735838e+06
17    5.656545e+05
18    1.100017e+06
19    3.648812e+05
20    1.147978e+06
21    1.388378e+06
22    1.200184e+06
23    1.404881e+06
24    3.603640e+06
dtype: float64

In [30]:
Pex_df = ext_pyomo_vals(model.Pex)

In [31]:
Pex_df

,0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0
10,0.0


,0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0
10,0.0


## Save results in csv files

In [32]:
import os 
folder = 'results_RES_b_' + str(n_evs)

if not os.path.exists(folder):
    os.makedirs(folder)
    
EEV_df.to_csv(folder + '/EEV.csv')
PEV_df.to_csv(folder + '/PEV.csv')
PEV_df.sum().to_csv(folder + '/PEV_h.csv')

Etriprelax_df.to_csv(folder + '/Etriprelax.csv')
Etriprelax_df.sum().to_csv(folder + '/Etriprelax_h.csv')

Eminsocrelax_df.to_csv(folder + '/Eminsocrelax.csv')
Eminsocrelax_df.sum().to_csv(folder + '/Eminsocrelax_h.csv')

Etripn_df.to_csv(folder + '/Etripn.csv')
Etripn_df.sum().to_csv(folder + '/Etripn_h.csv')

Pgen_df.to_csv(folder + '/Pgen.csv')
Pres_df.to_csv(folder + '/Pres.csv')
Pex_df.to_csv(folder + '/Pex.csv')